In [13]:
import pandas as pd
import os
import csv
import json
""" Replace 14 with 15 in full file, select proper log file"""
data_dir = '../code/data_dkt/delft14_phase1/'
prev_dir='../data/dktData/delft14/'
# event_log = os.path.join('../data'+'/DelftX_AE1110x_2T2015-events.log')
event_log = os.path.join('../data'+'/DelftX_AE1110x_1T2014-events.log')
import datetime
import pandas as pd
import json
import numpy as np
"""read skill dict file delft aerospace course data"""
problem_id_dict=json.load(open('data_dkt/delft15_phase2/skill_dict_delft_all_0_336_all.json'))
def process_time(time):
    return datetime.datetime.strptime(time[:-6], '%Y-%m-%dT%H:%M:%S.%f' if '.' in time[:-6] else '%Y-%m-%dT%H:%M:%S')

In [2]:
!pwd

/home/sanaiqbal/moocadaptive/code


In [2]:
!ls ../data/dktData/delft15/

delft_15_DKTrequired_data.csv	      response_df_delft_15.csv
delft_15_only_problem_check_log.json  skill_df_delft_15.csv


In [8]:

"""This will read log data and save problemcheck log events only"""
def read_raw_log_file(log_file=event_log):

    working_data = []
    fail_count = 0
    problem_check_count=0
    with open(log_file) as data_file:
        for line in data_file.readlines():
            try:
                data = json.loads(line)
                if data['event_type']=='problem_check':
                    working_data.append(data)

                    problem_check_count+=1
    #                     working_data.append(line)
            except:
                fail_count+=1
                print("Failed to read line:")
                print(line)
                continue
    print("Number of problem check log entries:", problem_check_count)
    print("Number of failed lines: ", fail_count)
    print("Length of successfuly read data: ", len(working_data))
    # SAVE LOG DATA  OF PROBLEM CHECK STUDENTS in a json file
    with open(data_dir+"delft_15_only_problem_check_log.json", 'w') as fout:
        json.dump(working_data, fout)
    return working_data
# working_data=read_raw_log_file()

In [9]:
"""load only the problem check log data if the problem check log does nt exist """
#  run the above function: read_raw_log_file()
# import json
# data_problem_check = json.load(open(data_dir+'delft_15_only_problem_check_log.json'))

'load only the problem check log data if the problem check log does nt exist '

In [19]:
'''#SAVE STUDENT DATA TO DATAFRAME '''

# listOfRows=[]
# for row in data_problem_check:
#     rowdict={}
#     rowdict["time"]=process_time(row['time'])
#     rowdict["username"]=row['username']
# """for 2015 edit problemid using this"""
##    rowdict['problem_id']=row['event']['problem_id'].split("@")[-1])
#     rowdict["problem_id"]=row['event']['problem_id'].split("/")[-1]
#     rowdict["success"]=row['event']['success']
#     rowdict["attempts"]=row['event']['attempts']
#     listOfRows.append(rowdict)
  
    
# df = pd.DataFrame(listOfRows)
# #df.to_csv(data_dir+'delft_14_DKT_problem_data_phase1.csv',index=False)


# start here

In [20]:
"""Read saved file of problem check events only"""
df=pd.read_csv(data_dir+'delft_14_DKT_problem_data_phase1.csv')
df.head()


,attempts,problem_id,success,time,username
0,1,b8ca0effd8db42dba28d14a90cbc1163,correct,2014-02-18 21:23:44.112191,username_118692035
1,1,5a06f0135c9c46a1843767519726753c,correct,2014-02-18 21:44:42.479603,username_118692035
2,2,6057ed87052c4aa3835ad5e30b91863d,incorrect,2014-02-19 22:00:47.645581,username_118692035
3,1,6057ed87052c4aa3835ad5e30b91863d,incorrect,2014-02-19 22:00:40.603703,username_118692035
4,11,6057ed87052c4aa3835ad5e30b91863d,correct,2014-02-19 22:04:01.605071,username_118692035


In [21]:
df.shape

(1041217, 5)

In [22]:
unique_user_id =set(filter(lambda x: x is not None, df['username'].tolist()))

In [23]:
len(unique_user_id)

5616

In [10]:
# get idea about number of timesteps for each users
df.groupby('username')['time'].count().describe()

count    9242.000000
mean       73.936269
std       148.752446
min         1.000000
25%         5.000000
50%        15.000000
75%        41.000000
max       845.000000
Name: time, dtype: float64

In [24]:
"""CONVERT TIME TO TIMESTEPS"""
def create_timesteps_truncated(group,limit=100):
#     print(group)
    group=group.sort_values('time')
#     assign timesteps from 1..
    indices=list(range(1,len(group["time"])+1))
    length=len(indices)
    if length>limit:
#         print(length)
#         set all other timesteps to 0
        indices[limit:]=[0]*(length-limit)
    group["timesteps"]=indices
    return group


df=df.groupby("username").apply(lambda group : create_timesteps_truncated(group))


""" Also we can create more datapoints by 
breaking large sequences into small length 
i.e multiple sequences per student """

' Also we can create more datapoints by \nbreaking large sequences into small length \ni.e multiple sequences per student '

In [25]:
df.timesteps.describe()

count    1.041217e+06
mean     1.025763e+01
std      2.268995e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.000000e+02
Name: timesteps, dtype: float64

In [26]:
# get rows where timstep is valid 
dfvalid=df[df['timesteps']!=0]
dfvalid.shape


(265233, 6)

In [27]:
dfvalid.head()

attempts                        problem_id  \
username                                                               
username_100663452 40881         1  b8ca0effd8db42dba28d14a90cbc1163   
                   40880         2  b8ca0effd8db42dba28d14a90cbc1163   
                   40932         1  5a06f0135c9c46a1843767519726753c   
                   40905         2  5a06f0135c9c46a1843767519726753c   
                   40891         3  5a06f0135c9c46a1843767519726753c   

                            success                        time  \
username                                                          
username_100663452 40881  incorrect  2014-03-04 02:22:09.491919   
                   40880  incorrect  2014-03-04 02:22:13.531655   
                   40932  incorrect  2014-03-04 02:26:46.615944   
                   40905  incorrect  2014-03-04 02:26:52.340584   
                   40891  incorrect  2014-03-04 02:26:55.772529   

                                    username  timesteps  
username                                                 
username_100663452 40881  username_100663452          1  
                   40880  username_100663452          2  
                   40932  username_100663452          3  
                   40905  username_100663452          4  
                   40891  username_100663452          5

In [28]:
assistment_df= dfvalid.pivot(index='username', columns='timesteps', values='problem_id')

In [ ]:
assistment_df.shape

In [ ]:
# assistment_df.isnull().sum()

### Replace id with integer numbers

In [29]:

skill_df=assistment_df.replace(problem_id_dict) 
skill_df=skill_df.fillna(-1)
skill_df=skill_df.astype(int)

In [18]:
skill_df.shape

(9242, 100)

In [19]:
skill_df.head()

timesteps,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
username,,,,,,,,,,,,,,,,,,,,,
username_100663505,18,327,51,168,302,302,152,44,322,244,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
username_100664898,281,14,14,14,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
username_100666909,281,281,14,14,14,91,91,91,305,305,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
username_100668446,281,14,14,14,314,314,314,314,91,91,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
username_100670598,281,14,91,91,91,314,314,305,305,305,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
# skill_df.isnull().sum()

In [30]:

#skill_df.to_csv(data_dir+'skill_df_delft_14_phase1.csv')

In [31]:
response_data= dfvalid.pivot(index='username', columns='timesteps', values='success')

In [32]:
# response_data.isnull().sum()
response_data.shape

(5616, 100)

In [33]:
correctness={"correct":int(1),"incorrect":int(0)}
response_df=response_data.replace(correctness) 

In [ ]:
# response_df.head()

In [34]:
response_df=response_df.fillna(-1)
response_df=response_df.astype(int)

In [35]:
# response_df.to_csv(data_dir+'response_df_delft_14_phase1.csv')